In [ ]:
# dependencies import
from common_dependencies import *
logger = logging.getLogger('main.model_test')

# Загрузка модели для тестирования

In [ ]:
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)
np.random.seed(290)

## Поиск модели по идентификатору

In [ ]:
MODEL_VER = 14
MODEL_NUM = 2

model_pathes = [path for path in pathlib.Path(f'networks/CNN/').rglob(f'*.keras') 
                     if re.search(rf'id=v0*{MODEL_VER}n0*{MODEL_NUM}',path.name)]
if len(model_pathes) == 1:
    PATH_TO_MODEL = model_pathes[0]
else:
    print(model_pathes)
    raise ValueError('Few or none model have been found instead of one')

print(f'{PATH_TO_MODEL=}')

## Загрузка модели

In [ ]:
model = keras.models.load_model(PATH_TO_MODEL)

print(model.summary())

tf.keras.utils.plot_model(
    model,
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=200,
    show_layer_activations=False,
    show_trainable=False,
)

# Загрузка данных для тестирования

## Параметры  выборок

In [ ]:
XSHIFT = 200
MAX_VAL = 1000
CROP_SIZE = model.inputs[0].shape[1]
dataset_desc =  {'test': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,80,MAX_VAL,MAX_VAL),SlidingCrop(CROP_SIZE,1),XSHIFT),
                          DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,80,MAX_VAL,MAX_VAL),SlidingCrop(CROP_SIZE,1),XSHIFT)
                            ,)}

## Загрузка данных

In [ ]:
dataset = {'test':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['test'])]))}

## Размерности данных в обучающих выборках

In [ ]:
logger.debug('Dataset')
for dataset_part_name, dataset_part in dataset.items():
    logger.debug('|'*8+dataset_part_name+'|'*8)
    for data_part_name, data_part in dataset_part.items():
        logger.debug(f'{data_part_name}.shape: {data_part.shape}')

## Какие части 2 файлов с данными относятся к конкретным выборкам (к тренировочной, тестовой, валидационной)

In [ ]:
#выборка данных
# show parts took for learning
all_rects = {'run_1': {'test':None,},  #
             'run_2': {'test':None,}} #
rects_colors = {'train':'red', 'val':'green', 'test':'yellow'}

for run_name in all_rects.keys():
    x_df, y_df = dw.get_x_and_y_data(*PATH_TO_DATA[run_name])
    x_df = None
    y_df = dw.roll_df(y_df, XSHIFT, 1)
    for dataset_part_name in all_rects[run_name].keys():
        # get all DatasetPartDescription for train, val or test
        dataset_part_desc = dataset_desc[dataset_part_name]
        # get all DatasetPartDescription for current run_name (run_1 or run_2)
        dataset_part_desc = [dataset_part for dataset_part in dataset_part_desc if re.findall(r'run_\d', dataset_part.data_path_tuple[0])[0] == run_name]
        # put rects list to all_rects[run_name][dataset_part_name]
        all_rects[run_name][dataset_part_name] = [Rectangle((dataset_part.file_data_crop.left, dataset_part.file_data_crop.top), 
                           dataset_part.file_data_crop.width, dataset_part.file_data_crop.height, 
                           facecolor=rects_colors[dataset_part_name], alpha=0.4, edgecolor='white')
                                                  for dataset_part in dataset_part_desc]
        
    res_rects = list(itertools.chain(*[run_rects for run_rects_name, run_rects in all_rects[run_name].items()]))
    if res_rects:
        dw.draw_defects_map_with_rectangles_owerlap(y_df, res_rects, title = f'The parts took for learning from {run_name} (red - train, green - validate, yellow - test)')
    else:
        dw.draw_defects_map(y_df, title = f'The parts took for learning from {run_name} (red - train, green - validate, yellow - test)')

# Тестирование модели

In [ ]:
res = model.evaluate(dataset['test']['x'], 
                     dataset['test']['bin'], 
                     batch_size=32, 
                     verbose=0, return_dict=True)

In [ ]:
print('|'*8,'test','|'*8)#,'\n',train_res,'\n')
print(*[(key,value) for (key,value) in res.items()],sep='\n')

In [ ]:
#dataset = {'test':dict(zip(['x','y'], [dw.get_batch_generator(part,5000) for part in chain_dataset_gens(dataset_desc['test'])] ))}

In [ ]:
'''metric_dict_14 = {'train':None,'val':None,'test':None}
for dataset_part_name, dataset_part in dataset.items():
    print('|'*8+dataset_part_name+'|'*8)
    metrics = [keras.metrics.AUC(name='PR_AUC', curve='PR'),
               keras.metrics.AUC(name='ROC_AUC', curve='ROC'),
               keras.metrics.BinaryCrossentropy(name='BinaryCrossentropy'),
               keras.metrics.BinaryAccuracy(name='BinaryAccuracy'),
               keras.metrics.MeanSquaredError(name='MeanSquaredError'),
               keras.metrics.Precision(name='Precision'),
               keras.metrics.Recall(name='Recall'),
               keras.metrics.TruePositives(name='TruePositives'),
               keras.metrics.FalsePositives(name='FalsePositives'),
               keras.metrics.TrueNegatives(name='TrueNegatives'),
               keras.metrics.FalseNegatives(name='FalseNegatives')]

    for (x_buffer,  y_buffer) in zip(*dataset_part.values()):
        x_buffer = np.pad(x_buffer, ((0,0),(1,1),(1,1),(0,0)), 'reflect')
        #print(x_buffer.shape)
        res = load_model.predict(x_buffer, verbose=0)
        for metric in metrics:
            metric.update_state(y_buffer, res)

    metric_dict_14[dataset_part_name] = {metric.get_config()["name"]:metric.result() for metric in metrics}
    for metric in metrics:
        #print(f'{metric.get_config()["name"]}:{metric.result():.6f}')
        metric.reset_state()''';

In [ ]:
'''CROP_SIZE = 16

dataset_desc =  {'train': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,0,MAX_VAL,MAX_VAL),SlidingCrop(CROP_SIZE,1),XSHIFT),),
                 'val': (DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,0,MAX_VAL,40),SlidingCrop(CROP_SIZE,1),XSHIFT),),
                 'test': (DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,40,MAX_VAL,MAX_VAL),SlidingCrop(CROP_SIZE,1),XSHIFT),)}

dataset = {'train':dict(zip(['x','y'], [dw.get_batch_generator(part,5000) for part in chain_dataset_gens(dataset_desc['train'])] )),
           'val':dict(zip(['x','y'], [dw.get_batch_generator(part,5000) for part in chain_dataset_gens(dataset_desc['val'])] )),
           'test':dict(zip(['x','y'], [dw.get_batch_generator(part,5000) for part in chain_dataset_gens(dataset_desc['test'])] ))}''';

In [ ]:
'''metric_dict_16 = {'train':None,'val':None,'test':None}
for dataset_part_name, dataset_part in dataset.items():
    print('|'*8+dataset_part_name+'|'*8)
    metrics = [keras.metrics.AUC(name='PR_AUC', curve='PR'),
               keras.metrics.AUC(name='ROC_AUC', curve='ROC'),
               keras.metrics.BinaryCrossentropy(name='BinaryCrossentropy'),
               keras.metrics.BinaryAccuracy(name='BinaryAccuracy'),
               keras.metrics.MeanSquaredError(name='MeanSquaredError'),
               keras.metrics.Precision(name='Precision'),
               keras.metrics.Recall(name='Recall'),
               keras.metrics.TruePositives(name='TruePositives'),
               keras.metrics.FalsePositives(name='FalsePositives'),
               keras.metrics.TrueNegatives(name='TrueNegatives'),
               keras.metrics.FalseNegatives(name='FalseNegatives')]

    for (x_buffer,  y_buffer) in zip(*dataset_part.values()):
        #x_buffer = np.pad(x_buffer, ((0,0),(1,1),(1,1),(0,0)), 'reflect')
        #print(x_buffer.shape)
        res = load_model.predict(x_buffer, verbose=0)
        for metric in metrics:
            metric.update_state(y_buffer, res)

    metric_dict_16[dataset_part_name] = {metric.get_config()["name"]:metric.result() for metric in metrics}
    for metric in metrics:
        #print(f'{metric.get_config()["name"]}:{metric.result():.6f}')
        metric.reset_state()''';

In [ ]:
'''for (data_part1, data_part2) in zip(metric_dict_14.keys(), metric_dict_16.keys()):
    print('|'*8+data_part1+'|'*8)
    for ((metric_name1, metric_res1), (metric_name2, metric_res2)) in zip(metric_dict_14[data_part1].items(), metric_dict_16[data_part2].items()):
        print('|'*8+metric_name1+'|'*8)
        print(f'Вход 16 на 16: {metric_res2}')
        print(f'Вход 14 на 14: {metric_res1}')
        print(f'Метрика для 14 на 14 это {metric_res1/metric_res2*100:.2f}% от метрики 16 на 16')
        print()''';

In [ ]:
'''# тест модели
res_list = list()
for crop_step in CROP_STEPS:
    (test_x_time_gen, 
     test_x_amp_gen, 
     test_y_binary_gen,
     test_y_depth_gen) = get_dataset_gen(PATH_TO_DATA[f'run_{run}'], XSHIFT, DataCrop(None,None,None,200), CROP_SIZE, 20)
    
    test_x_time = np.stack([crop for crop in test_x_time_gen])
    test_x_amp = np.stack([crop for crop in test_x_amp_gen])
    test_y_binary = np.array([binary for binary in test_y_binary_gen])
    test_y_depth = np.array([depth for depth in test_y_depth_gen])
    
    res = load_model.evaluate([test_x_time, test_x_amp], test_y_binary, batch_size = 32, return_dict=True)
    res_list.append({'res':res,'crop_size':CROP_SIZE,'crop_step':crop_step})''';

In [ ]:
'''for line in res_list:
    print(f'crop_size: {line["crop_size"]}')
    print(f'crop_size: {line["crop_step"]}')
    print(line['res'])
    print()''';